In this step we are filtering the tweets of the Key Opinion Leaders, that we retrived in the first step. 


In [1]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [4]:
import pandas as pd

tweets_table_filtered = pd.read_excel('tweets_table_filtered.xlsx')
tweets_table_filtered['conversation_id'] = tweets_table_filtered['conv_id_new'].apply(lambda x: x.replace("conv_id: ", ""))
tweets_table_filtered = tweets_table_filtered.add_prefix('KOL_Tweet_')


In [6]:
top_n_tweets = list(tweets_table_filtered.head()['KOL_Tweet_conversation_id'])
top_n_tweets

['764361626212524032',
 '1164940747847667714',
 '1076417292139982849',
 '859821415675174912',
 '702594185942188033']

## Comments

In [7]:
path_for_table_dict = my_twitter_crawler.return_comments_by_tweet_ids(conversation_ids = top_n_tweets, query = "",
                              start_time = "2014-01-01T00:00:00Z",
                              end_time = "today",
                              max_results = 100, evaluate_last_token = False,
                              limit_amount_of_returned_comments = 500000,
                              verbose = False, dir_tree_name = "Step 3 Comments")

Bringing comments of 764361626212524032
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_764361626212524032 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_764361626212524032\log_comments_for_conversation_id_764361626212524032 to insert all the logs of the comments for the tweet id -  764361626212524032
no data / include in the json
no more comments from this conversation id
Total amount of collected comments =  0
There was a problem with the conversation id: 764361626212524032
*************************************************************************************
Bringing comments of 1164940747847667714
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_1164940747847667714 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_1164940747847667714\log_comments_for_conversation_id_1164940747847667714 to insert 

In [16]:
TwitterCrawler.get_url_by_tweet_id(tweet_id = "1072239308554362881")

'https://twitter.com/anyuser/status/1072239308554362881'

In [8]:
import os
 
directory = 'Step 3 Comments'

all_comments_df_list = []
for root, dirs, files in os.walk(directory):
    for filename in files:
        if not filename.endswith('csv'): continue
        file_path = os.path.join(root, filename)
        comments_table =  pd.read_csv(file_path)
        all_comments_df_list.append( comments_table)

all_comments_df = pd.concat(all_comments_df_list)
all_comments_df['conversation_id'] = all_comments_df['conversation_id'].apply(lambda x : str(x))
all_comments_df = all_comments_df.add_prefix('Comment_')

In [9]:

all_comments_df

,Comment_Unnamed: 0,Comment_author_id,Comment_author_id_new,Comment_conv_id_new,Comment_conversation_id,Comment_created_at,Comment_entities.mentions,Comment_id,Comment_id_new,Comment_in_reply_to_user_id,...,Comment_users.created_at,Comment_users.description,Comment_users.id,Comment_users.name,Comment_users.public_metrics.followers_count,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,0,213044641,author_id: 213044641,coversation_id: 1072239308554362881,1072239308554362881,2018-12-18T12:42:57.000Z,"[{'start': 0, 'end': 12, 'username': 'anthonyz...",1075008513909366785,id: 1075008513909366785,15662428,...,2010-11-07T20:19:36.000Z,Economics: The Truth Hurts,213044641,eh? eh?,87,659,3,17623,pdboxer,False
1,1,92599000,author_id: 92599000,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T08:32:24.000Z,"[{'start': 0, 'end': 16, 'username': 'charles_...",1073858301102866432,id: 1073858301102866432,14292921,...,2009-11-25T19:59:31.000Z,"Cardiff Devils hockey & Wales rugby fan, Cymro...",92599000,Hedd Wyn John 🏴󠁧󠁢󠁷󠁬󠁳󠁿,782,1252,5,24110,Hedd_Wyn_John,False
2,2,795921978758729728,author_id: 795921978758729728,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T07:40:53.000Z,"[{'start': 0, 'end': 12, 'username': 'anthonyz...",1073845334391050240,id: 1073845334391050240,15662428,...,2016-11-08T09:32:42.000Z,Virtuous pagan. Very proud to be British. Love...,795921978758729728,🇬🇧George Lynes🇮🇹,92,12,0,7123,LynesGeorge,False
3,3,14292921,author_id: 14292921,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T02:16:03.000Z,"[{'start': 0, 'end': 15, 'username': 'danielmg...",1073763587729080321,id: 1073763587729080321,938591299,...,2008-04-03T13:00:45.000Z,The Humbled Programmer near the Minster,14292921,Charles Forsyth,654,441,22,22035,charles_forsyth,False
4,4,14292921,author_id: 14292921,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T02:13:40.000Z,"[{'start': 0, 'end': 15, 'username': 'danielmg...",1073762986353926145,id: 1073762986353926145,14292921,...,2008-04-03T13:00:45.000Z,The Humbled Programmer near the Minster,14292921,Charles Forsyth,654,441,22,22035,charles_forsyth,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2,50394715,author_id: 50394715,coversation_id: 702594185942188033,702594185942188033,2016-02-24T20:49:36.000Z,"[{'start': 12, 'end': 21, 'username': 'Itwitiu...",702596260415586304,id: 702596260415586304,2653613168,...,2009-06-24T18:46:43.000Z,"IT, SelloTape, String, the usual.\nI sit slig...",50394715,AdmV0rl0n,905,2685,31,43025,AdmV0rl0n,False
102,3,2234063790,author_id: 2234063790,coversation_id: 702594185942188033,702594185942188033,2016-02-24T20:47:13.000Z,"[{'start': 62, 'end': 73, 'username': 'vote_le...",702595659405389828,id: 702595659405389828,2653613168,...,2013-12-07T06:22:28.000Z,Chair @ebbsfleetdc | Chair Square Roots RP | F...,2234063790,Simon Dudley,2640,500,53,13036,MrSimonDudley,False
103,4,1119758522,author_id: 1119758522,coversation_id: 702594185942188033,702594185942188033,2016-02-24T20:46:04.000Z,NaN,702595368626855936,id: 702595368626855936,2653613168,...,2013-01-25T16:43:52.000Z,"They lie for a living, it's what they do",1119758522,LIAR MPs,16322,1000,0,50009,LiarMPs,False
0,0,21111724,author_id: 21111724,coversation_id: 859743329952706560,859743329952706560,2017-05-03T12:45:53.000Z,"[{'start': 0, 'end': 14, 'username': 'MartinSe...",859750862180290561,id: 859750862180290561,2421073909,...,2009-02-17T18:00:24.000Z,Expect the unexpected. HÄRTING Rechtsanwälte s...,21111724,Niko Härting,16964,100,340,38133,nhaerting,False


In [10]:
## Merged Tweets with comments (each row is a tweet plus one of its comments, meaning that a tweet with 50 comments
# appears in 50 rows of the table
merged_comments = pd.merge(tweets_table_filtered,all_comments_df,left_on='KOL_Tweet_conversation_id',right_on='Comment_conversation_id' , how="inner")

In [11]:
merged_comments

,KOL_Tweet_Unnamed: 0,KOL_Tweet_Unnamed: 0.1,KOL_Tweet_author_id,KOL_Tweet_conv_id_new,KOL_Tweet_conversation_id,KOL_Tweet_created_at,KOL_Tweet_entities.mentions,KOL_Tweet_id,KOL_Tweet_id_new,KOL_Tweet_public_metrics.like_count,...,Comment_users.created_at,Comment_users.description,Comment_users.id,Comment_users.name,Comment_users.public_metrics.followers_count,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,38206,439,17487476,conv_id: 1164940747847667714,1164940747847667714,2019-08-23T16:41:32.000Z,NaN,1164940747847668000,id: 1164940747847667714,130,...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,26295559,Roeland van Zeijst,1617,3973,41,100542,rovaz,False
1,38206,439,17487476,conv_id: 1164940747847667714,1164940747847667714,2019-08-23T16:41:32.000Z,NaN,1164940747847668000,id: 1164940747847667714,130,...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,26295559,Roeland van Zeijst,1617,3973,41,100542,rovaz,False
2,38206,439,17487476,conv_id: 1164940747847667714,1164940747847667714,2019-08-23T16:41:32.000Z,NaN,1164940747847668000,id: 1164940747847667714,130,...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,26295559,Roeland van Zeijst,1617,3973,41,100542,rovaz,False
3,38206,439,17487476,conv_id: 1164940747847667714,1164940747847667714,2019-08-23T16:41:32.000Z,NaN,1164940747847668000,id: 1164940747847667714,130,...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,26295559,Roeland van Zeijst,1617,3973,41,100542,rovaz,False
4,38206,439,17487476,conv_id: 1164940747847667714,1164940747847667714,2019-08-23T16:41:32.000Z,NaN,1164940747847668000,id: 1164940747847667714,130,...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,26295559,Roeland van Zeijst,1617,3973,41,100542,rovaz,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,40969,265,2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,702594185942188000,id: 702594185942188033,106,...,2009-06-24T18:46:43.000Z,"IT, SelloTape, String, the usual.\nI sit slig...",50394715,AdmV0rl0n,905,2685,31,43025,AdmV0rl0n,False
196,40969,265,2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,702594185942188000,id: 702594185942188033,106,...,2013-12-07T06:22:28.000Z,Chair @ebbsfleetdc | Chair Square Roots RP | F...,2234063790,Simon Dudley,2640,500,53,13036,MrSimonDudley,False
197,40969,265,2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,702594185942188000,id: 702594185942188033,106,...,2013-01-25T16:43:52.000Z,"They lie for a living, it's what they do",1119758522,LIAR MPs,16322,1000,0,50009,LiarMPs,False
198,30955,359,2421073909,conv_id: 859743329952706560,859743329952706560,2017-05-03T12:19:37.000Z,"[{'start': 0, 'end': 10, 'username': 'nhaertin...",859744249780338700,id: 859744249780338688,1,...,2009-02-17T18:00:24.000Z,Expect the unexpected. HÄRTING Rechtsanwälte s...,21111724,Niko Härting,16964,100,340,38133,nhaerting,False


In [92]:
merged_comments.columns

Index(['KOL_Tweet_Unnamed: 0', 'KOL_Tweet_Unnamed: 0.1', 'KOL_Tweet_author_id',
       'KOL_Tweet_conv_id_new', 'KOL_Tweet_conversation_id',
       'KOL_Tweet_created_at', 'KOL_Tweet_entities.mentions', 'KOL_Tweet_id',
       'KOL_Tweet_id_new', 'KOL_Tweet_public_metrics.like_count',
       'KOL_Tweet_public_metrics.quote_count',
       'KOL_Tweet_public_metrics.reply_count',
       'KOL_Tweet_public_metrics.retweet_count', 'KOL_Tweet_referenced_tweets',
       'KOL_Tweet_text', 'KOL_Tweet_users.created_at',
       'KOL_Tweet_users.description', 'KOL_Tweet_users.id',
       'KOL_Tweet_users.name',
       'KOL_Tweet_users.public_metrics.followers_count',
       'KOL_Tweet_users.public_metrics.following_count',
       'KOL_Tweet_users.public_metrics.listed_count',
       'KOL_Tweet_users.public_metrics.tweet_count',
       'KOL_Tweet_users.username', 'KOL_Tweet_users.verified',
       'KOL_Tweet_text_tokens', 'KOL_Tweet_created_at_date',
       'KOL_Tweet_is_in_special_date', 'KOL_Tweet_

In [16]:
final_table = merged_comments[['KOL_Tweet_author_id', 'KOL_Tweet_users.name', 'KOL_Tweet_conversation_id', 'KOL_Tweet_text', 'KOL_Tweet_text_tokens', 
'Comment_author_id', 'Comment_text',
       'Comment_users.created_at', 'Comment_users.description',
'Comment_users.public_metrics.following_count',
       'Comment_users.public_metrics.listed_count',
       'Comment_users.public_metrics.tweet_count', 'Comment_users.username',
       'Comment_users.verified' ]]

#final_table['KOL ID'] = final_table['KOL ID'].apply(lambda x: x.replace("author_id: ", ""))
#final_table['Commenter ID'] = final_table['Commenter ID'].apply(lambda x: x.replace("author_id: ", ""))

final_table

,KOL_Tweet_author_id,KOL_Tweet_users.name,KOL_Tweet_conversation_id,KOL_Tweet_text,KOL_Tweet_text_tokens,Comment_author_id,Comment_text,Comment_users.created_at,Comment_users.description,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@cswheatley @MichelBarnier You think the UK is...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
1,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@YusufCilek3 @MichelBarnier If you have the ti...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
2,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@patrick00054344 @MichelBarnier @MauriceMartin...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
3,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@WTO_Rules @emeraldeagle2 @MichelBarnier Objec...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
4,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@Stuartcassidy9 @MichelBarnier Ciao is actuall...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2653613168,Philip Hammond,702594185942188033,Britain's #EU deal has been registered at the ...,"['britain', 'eu', 'deal', 'registered', 'un', ...",50394715,@PHammondMP @Itwitius Its time to leave.,2009-06-24T18:46:43.000Z,"IT, SelloTape, String, the usual.\nI sit slig...",2685,31,43025,AdmV0rl0n,False
196,2653613168,Philip Hammond,702594185942188033,Britain's #EU deal has been registered at the ...,"['britain', 'eu', 'deal', 'registered', 'un', ...",2234063790,@PHammondMP Our Justice Secretary Michael Gove...,2013-12-07T06:22:28.000Z,Chair @ebbsfleetdc | Chair Square Roots RP | F...,500,53,13036,MrSimonDudley,False
197,2653613168,Philip Hammond,702594185942188033,Britain's #EU deal has been registered at the ...,"['britain', 'eu', 'deal', 'registered', 'un', ...",1119758522,@PHammondMP It's the deal that's not a deal. N...,2013-01-25T16:43:52.000Z,"They lie for a living, it's what they do",1000,0,50009,LiarMPs,False
198,2421073909,Martin Selmayr,859743329952706560,@nhaerting @VivianeRedingEU Und heute steht di...,"['nhaerting', 'vivianeredingeu', 'und', 'heute...",21111724,@MartinSelmayr @VivianeRedingEU Wir Berater si...,2009-02-17T18:00:24.000Z,Expect the unexpected. HÄRTING Rechtsanwälte s...,100,340,38133,nhaerting,False
